# Part II - VacationPy

### Now let's use your skills in working with weather data to plan future vacations. Use jupyter-gmaps and the Google Places API for this part of the assignment.

In [1]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
import gmaps
import os

# Import API key (G Map)
from config import gkey

In [2]:
weather_data = pd.read_csv("../WeatherPy/output_data/weather_cities.csv")

print(f'There are {len(weather_data)} cities in this dataset.')

weather_data.head()

There are 558 cities in this dataset.


,Cities,Country codes,Latitudes,Longitudes,Temperatures (Fº),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,Kaitangata,NZ,-46.28,169.85,43.00,76,41,1.99
1,Inírida,CO,3.87,-67.92,71.44,97,98,2.08
2,Ponta do Sol,PT,32.67,-17.10,67.35,64,40,11.41
3,Otane,NZ,-39.88,176.63,52.00,88,80,10.00
4,Yoichi,JP,43.20,140.77,62.94,77,75,14.99


### Heatmap
 - Create a heat map that displays the humidity for every city from the part I of the homework.

In [3]:
# Create a heat map that displays the humidity for every city from the part I of the homework.
# file Banking_Desert_HeatMap.ipynb

# Configure gmaps with API key
gmaps.configure(api_key=gkey)

# Store 'Lat' and 'Lng' into locations
locations = weather_data[["Latitudes", "Longitudes"]].astype(float)
humidity = weather_data["Humidity (%)"].astype(float)

# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Narrow down the DataFrame to find your ideal weather condition.
For example,
 - A max temperature lower than or equal to 80 degrees but higher than or equal to 70.
 - Wind speed less than or equal to 10 mph.
 - Zero cloudiness.

Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

In [4]:
# Extract the "ideal conditions" data
ideal_conditions = weather_data.loc[(weather_data["Temperatures (Fº)"] >= 70) & 
                                    (weather_data["Temperatures (Fº)"] <= 80) &
                                    (weather_data["Wind Speed (mph)"] <= 10) & 
                                    (weather_data["Cloudiness (%)"] == 0)].dropna()

hotel_df = ideal_conditions.reset_index(drop=True)

print(f'There are {len(hotel_df)} cities in the ideal conditions in this dataset.')

hotel_df.head()

There are 17 cities in the ideal conditions in this dataset.


,Cities,Country codes,Latitudes,Longitudes,Temperatures (Fº),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,Altar,MX,30.70,-111.82,77.40,17,0,3.42
1,Al Bardīyah,LY,31.76,25.09,78.76,55,0,9.71
2,Sirte,LY,31.21,16.59,76.86,64,0,7.47
3,Cabo San Lucas,MX,22.89,-109.91,72.36,86,0,8.08
4,Roebourne,AU,-20.78,117.13,76.14,30,0,8.39


### Find Hotels and Plot the Hotels

In [5]:
# Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
# files Google_Places.ipynb, Google_That.ipynb; NearestRestr.ipynb, Airport_Ratings.ipynb

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# set up a parameters dictionary
params = {
    #"location": target_coordinates,
    "keyword": "Hotel",
    "radius": 5000,
    "key": gkey
}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in hotel_df.iterrows():

    lat = row['Latitudes']
    lng = row['Longitudes']
    city_name = row["Cities"]

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

     # update address key value
    params['location'] = f"{lat},{lng}" # not 'locations'

    # make request
    response = requests.get(base_url, params=params).json()
    
    
    # save the hotel name to dataframe
    try:
        #hotel_df.loc[index, "Latitudes"] = response["results"][0]["geometry"]["location"]["lat"]
        #hotel_df.loc[index, "Longitudes"] = response["results"][0]["geometry"]["location"]["lng"]
        print(f"The closest hotel in {city_name} is {response['results'][0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = response['results'][0]['name']
        #IndexError: list index out of range

    # if there is no hotel available, show missing field
    except (IndexError):
        print(f'Oops, the closest hotel is not found in {city_name}.')

# Visualize to confirm lat lng appear
hotel_df.head()

The closest hotel in Altar is Hotel San Angel.
The closest hotel in Al Bardīyah is Hotel Burdy.
The closest hotel in Sirte is City Hotel.
The closest hotel in Cabo San Lucas is Waldorf Astoria Los Cabos Pedregal.
The closest hotel in Roebourne is Latitude20 Roebourne Village.
The closest hotel in Forsytheganj is Hotel Svetlogorsk JBIK.
The closest hotel in Altea is SH Villa Gadea.
Oops, the closest hotel is not found in Zhigansk.
The closest hotel in Tarasovskiy is Ariel-D.
Oops, the closest hotel is not found in Sobolevo.
Oops, the closest hotel is not found in Uspenka.
The closest hotel in Gifhorn is Deutsches Haus.
Oops, the closest hotel is not found in Lyuban'.
Oops, the closest hotel is not found in Namwala.
Oops, the closest hotel is not found in Nokaneng.
The closest hotel in Almaznyy is Gostinitsa Zverevo.
The closest hotel in Lavumisa is Royal Jozini Big 6 Private Estate.


,Cities,Country codes,Latitudes,Longitudes,Temperatures (Fº),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
0,Altar,MX,30.70,-111.82,77.40,17,0,3.42,Hotel San Angel
1,Al Bardīyah,LY,31.76,25.09,78.76,55,0,9.71,Hotel Burdy
2,Sirte,LY,31.21,16.59,76.86,64,0,7.47,City Hotel
3,Cabo San Lucas,MX,22.89,-109.91,72.36,86,0,8.08,Waldorf Astoria Los Cabos Pedregal
4,Roebourne,AU,-20.78,117.13,76.14,30,0,8.39,Latitude20 Roebourne Village


In [6]:
# NOTE: Do not change any of the code in this cell -> DO change (columns have different names)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{Cities}</dd>
<dt>Country</dt><dd>{Country codes}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitudes", "Longitudes"]]

In [7]:
# Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

# Add marker layer and info box content ontop of heat map
hotel_markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(hotel_markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Create a hotel symbol layer

# Convert hotels to a list
hotels_list = hotel_df["Hotel Name"].tolist()

# Create a hotel symbol layer
hotel_symbol_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info)

fig.add_layer(hotel_symbol_layer)

fig

Figure(layout=FigureLayout(height='420px'))